# AED - Análise Exploratória de Dados WS (Wind Speed - Velocidade do Vento) - Diário

## 1. Imports e Carregamento de Dados

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
csv_path = "ws_days.csv"

df = pd.read_csv(csv_path, dtype={"date": str})
df["date"] = pd.to_datetime(df["date"], format="%Y%m%d")
df = df.rename(columns={"ws2m_mean": "ws"})
df = df.set_index("date").sort_index()

## 2. Preparação - Índice Contínuo e Interpolação

In [ ]:
full_idx = pd.date_range(df.index.min(), df.index.max(), freq="D")
df = df.reindex(full_idx)
# Interpolação linear para possíveis buracos
df["ws"] = df["ws"].interpolate(limit_direction="both")

## 3. Estatísticas Descritivas

In [ ]:
print("Resumo estatístico:")
print(df["ws"].describe(), "\n")

## 4. Verificação de Qualidade dos Dados

In [ ]:
missing = df["ws"].isna().sum()
print(f"Valores faltantes após interpolação: {missing}")

## 5. Visualizações Básicas

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(df.index, df["ws"], label="WS (m/s)")
plt.plot(df.index, df["ws"].rolling(7, min_periods=1).mean(), label="Média móvel 7d", linewidth=2)
plt.title("Velocidade do vento média diária (com média móvel 7d)")
plt.xlabel("Data"); plt.ylabel("m/s"); plt.legend()
plt.tight_layout()
plt.show()

## 6. Distribuição por Dia da Semana

In [ ]:
tmp = df.copy()
tmp["dow"] = tmp.index.dayofweek  # 0 = segunda ... 6 = domingo
# Para boxplot, montamos os dados por categoria de dow
data_by_dow = [tmp.loc[tmp["dow"]==d, "ws"].values for d in range(7)]
plt.figure(figsize=(8,4))
plt.boxplot(data_by_dow, labels=["Seg","Ter","Qua","Qui","Sex","Sáb","Dom"], showmeans=True)
plt.title("Distribuição por dia da semana (sazonalidade semanal)")
plt.ylabel("m/s")
plt.tight_layout()
plt.show()

## 7. Decomposição STL (Semanal)

In [ ]:
stl = STL(df["ws"], period=7, robust=True)
res = stl.fit()

plt.figure(figsize=(10,8))
ax1 = plt.subplot(3,1,1); ax1.plot(res.trend); ax1.set_title("Tendência")
ax2 = plt.subplot(3,1,2); ax2.plot(res.seasonal); ax2.set_title("Sazonalidade (semanal)")
ax3 = plt.subplot(3,1,3); ax3.plot(res.resid); ax3.set_title("Resíduo")
plt.tight_layout()
plt.show()

## 8. Testes de Estacionariedade

In [ ]:
def adf_report(series, name="serie"):
    result = adfuller(series.dropna(), autolag="AIC")
    stat, pvalue, usedlag, nobs, crit, icbest = result
    print(f"=== ADF: {name} ===")
    print(f"Estatística: {stat:.3f} | p-valor: {pvalue:.4f} | lags: {usedlag} | n: {nobs}")
    print("Críticos:", crit, "\n")

adf_report(df["ws"], "WS (original)")
adf_report(res.resid, "resíduo STL")

## 9. ACF e PACF

In [ ]:
lags = min(30, len(df)-2)

plt.figure(figsize=(10,4))
plot_acf(df["ws"], lags=lags)
plt.title("ACF - Série original")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,4))
plot_acf(res.resid.dropna(), lags=lags)
plt.title("ACF - Resíduo STL")
plt.tight_layout()
plt.show()

## 10. Detecção de Outliers

In [ ]:
z = (df["ws"] - df["ws"].mean()) / df["ws"].std(ddof=0)
outliers = df[np.abs(z) > 3]
print(f"Nº de possíveis outliers (>|3σ|): {len(outliers)}")
if len(outliers):
    print(outliers)

## 11. Padrões Mensais

In [ ]:
tmp = df.copy()
tmp['month'] = tmp.index.month

plt.figure(figsize=(10,3.6))
(tmp.groupby('month')['ws'].mean()
 .rename(index={1:'Jan',2:'Fev',3:'Mar',4:'Abr',5:'Mai',6:'Jun',7:'Jul',8:'Ago',9:'Set',10:'Out',11:'Nov',12:'Dez'})
 .plot(kind='bar'))
plt.title('Velocidade média do vento por mês'); plt.ylabel('m/s'); plt.tight_layout(); plt.show()